# **Imports**🎇

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm
import os
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

import torch
from torch.utils.data import DataLoader, random_split, Dataset
import torch.optim as optim

import torchvision
from torchvision import transforms as T
from torchvision.utils import make_grid
from torchvision.transforms import functional as F

# **Utils**

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

n_epoch = 50

# **Prepare Labels and Files**

In [3]:
!labelme

2024-07-21 09:10:12,350 [INFO   ] __init__:get_config:67- Loading config file from: C:\Users\KARBAR\.labelmerc


# **Functions**

In [ ]:
class TrumpDataset(Dataset):
  def __init__(self, root, phase):
    self.root = root
    self.phase = phase

    # self.imgs = os.listdir(os.path.join(root, 'images'))
    self.targets = pd.read_csv(os.path.join(root, 'data/{}_labels.csv'.format(phase)))
    self.imgs = self.targets['filename']

  def __getitem__(self, idx):
    img_path = os.path.join(self.root, 'images', self.imgs[idx])
    img = Image.open(img_path).convert('RGB')
    img = F.to_tensor(img)

    #
    box_list = self.targets[self.targets['filename'] == self.imgs[idx]]
    box_list = box_list[['xmin', 'ymin', 'xmax', 'ymax']].values
    boxes = torch.tensor(box_list, dtype=torch.float32)
    #
    labels = torch.ones((len(box_list), ), dtype=torch.int64)
    #
    target = {}
    target['boxes'] = boxes
    target['labels'] = labels
    #
    return img, target

  def __len__(self):
    return len(self.imgs)